## 1. Balance Age groups in YLFW (undersampling)

In [100]:
import pandas as pd
import seaborn as sns
sns.set()
import random
random.seed(42)


### 1.1 Young Group

In [101]:
y_df = pd.read_csv('../data/raw_full/raw_ylfw_df.csv')

In [102]:
y_df[y_df.children_agegroup == "7-9"].groupby("im_id").agg({'img_name': ['count']}).describe()

,img_name
,count
count,955.000000
mean,1.732984
std,1.136820
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,11.000000


In [103]:
len(y_df)

9229

In [104]:
y_df.Age.describe()

count    9229.000000
mean        8.412829
std         7.504623
min         0.000000
25%         3.000000
50%         6.000000
75%        11.000000
max        42.000000
Name: Age, dtype: float64

In [105]:
y_df["children_agegroup"].value_counts()

children_agegroup
0-3      2665
4-6      2068
7-9      1655
10-12     923
13-15     459
16-18     383
Name: count, dtype: int64

### 1.2 Adult Group

In [106]:
a_df = pd.read_csv('../data/raw_full/raw_rfw_df.csv')

In [107]:
len(a_df)

40227

In [108]:
a_df

,img_name,im_id,ethnicity,id_class,face_id,Age,children_agegroup
0,m.05syl08_0003,m.05syl08,Indian,05syl08,3,64,NaN
1,m.05syl08_0002,m.05syl08,Indian,05syl08,2,62,NaN
2,m.05syl08_0001,m.05syl08,Indian,05syl08,1,64,NaN
3,m.05syl08_0004,m.05syl08,Indian,05syl08,4,61,NaN
4,m.0jzt0s_0002,m.0jzt0s,Indian,0jzt0s,2,53,NaN
...,...,...,...,...,...,...,...
40222,m.07rqxl_0002,m.07rqxl,Asian,07rqxl,2,44,NaN
40223,m.07rqxl_0001,m.07rqxl,Asian,07rqxl,1,49,NaN
40224,m.0d65nf_0002,m.0d65nf,Asian,0d65nf,2,53,NaN
40225,m.0d65nf_0003,m.0d65nf,Asian,0d65nf,3,57,NaN


In [109]:
a_df.Age.value_counts()

Age
26    1630
27    1614
25    1598
28    1496
24    1463
      ... 
2        2
3        2
85       2
88       1
90       1
Name: count, Length: 87, dtype: int64

In [110]:
# Add childrens age group to adult data
# Ensure 'Age' is of type integer
a_df['Age'] = a_df['Age'].astype(int)

# Define the age groups and corresponding labels
age_bins = [0, 4, 7, 10, 13, 16, 19]  # adding one more bin for 16-18
age_labels = ['0-3', '4-6', '7-9', '10-12', '13-15', '16-18']

# Update the DataFrame with age groupings
a_df['children_agegroup'] = pd.cut(a_df['Age'], bins=age_bins, labels=age_labels, right=False)
a_df["children_agegroup"].value_counts()


children_agegroup
16-18    452
13-15     92
10-12     82
7-9       43
4-6       24
0-3        4
Name: count, dtype: int64

In [111]:
a_df["children_agegroup"].isna().sum()

39530

### 3. Transfer childrens age groups in a_df to y_df

In [112]:
a_data_children_agegroup = a_df[a_df.children_agegroup.notnull()]

In [113]:
a_data_children_agegroup.groupby("im_id").agg({'img_name': ['count']}).value_counts()

(img_name, count)
1                    263
2                     81
3                     50
4                     25
5                      3
7                      1
Name: count, dtype: int64

In [114]:
# Merge to ylfw
ylfw_witha = pd.concat([y_df,a_data_children_agegroup],ignore_index=True)
ylfw_witha[-10:]

,img_name,Age,ethnicity,id_class,children_agegroup,im_id,face_id
9916,m.02qpxkm_0004,18,Asian,02qpxkm,16-18,m.02qpxkm,4.0
9917,m.01m49_z_0001,14,Asian,01m49_z,13-15,m.01m49_z,1.0
9918,m.0sgqn7h_0003,9,Asian,0sgqn7h,7-9,m.0sgqn7h,3.0
9919,m.0sgqn7h_0002,17,Asian,0sgqn7h,16-18,m.0sgqn7h,2.0
9920,m.0knwxrc_0001,16,Asian,0knwxrc,16-18,m.0knwxrc,1.0
9921,m.0d5060_0004,18,Asian,0d5060,16-18,m.0d5060,4.0
9922,m.0d5060_0002,18,Asian,0d5060,16-18,m.0d5060,2.0
9923,m.05mz42q_0002,18,Asian,05mz42q,16-18,m.05mz42q,2.0
9924,m.0rpfv5x_0004,17,Asian,0rpfv5x,16-18,m.0rpfv5x,4.0
9925,m.048t9m_0002,18,Asian,048t9m,16-18,m.048t9m,2.0


In [115]:
ylfw_witha

,img_name,Age,ethnicity,id_class,children_agegroup,im_id,face_id
0,Caucasian_1008_18,8,Caucasian,1008,7-9,Caucasian_1008,NaN
1,Asian_225_72,11,Asian,225,10-12,Asian_225,NaN
2,Asian_238_1,3,Asian,238,0-3,Asian_238,NaN
3,Caucasian_1110_17,4,Caucasian,1110,4-6,Caucasian_1110,NaN
4,Caucasian_364_12,2,Caucasian,364,0-3,Caucasian_364,NaN
...,...,...,...,...,...,...,...
9921,m.0d5060_0004,18,Asian,0d5060,16-18,m.0d5060,4.0
9922,m.0d5060_0002,18,Asian,0d5060,16-18,m.0d5060,2.0
9923,m.05mz42q_0002,18,Asian,05mz42q,16-18,m.05mz42q,2.0
9924,m.0rpfv5x_0004,17,Asian,0rpfv5x,16-18,m.0rpfv5x,4.0


### 4. Undersample based on the minority class in the children's age group in YLFW
- Keep racial distribution as in minority class
- Delete based on IDs 

Distribution of age groups - based on IDS (obs be aware that each id have images in different age groups). Therefore we sample based on number of images per age group. 


In [116]:
# Standard deviation of age groups per identity
ylfw_witha.groupby("im_id").agg({'Age': ['min', 'max','std', 'count']}).sort_values(by=('Age', 'std'), ascending=False).head(5)

Age                     
           min max        std count
im_id                              
Indian_87    0  30  21.213203     2
Asian_620    2  29  19.091883     2
Asian_570    2  35  18.502252     3
Asian_96     0  26  18.384776     2
African_44   6  32  18.384776     2

In [117]:
# Distribution of IDs per age group
ylfw_witha.groupby('children_agegroup').im_id.nunique().sort_values(ascending=False)

children_agegroup
4-6      1169
0-3      1123
7-9       987
10-12     670
16-18     650
13-15     428
Name: im_id, dtype: int64

In [118]:
# Distribution of age groups - based on number of images per age group
ylfw_witha.children_agegroup.value_counts().sort_values(ascending=False)

children_agegroup
0-3      2669
4-6      2092
7-9      1698
10-12    1005
16-18     835
13-15     551
Name: count, dtype: int64

In [119]:
# Take minority age group BASED ON N_IMAGES
min_agegroup = ylfw_witha.groupby('children_agegroup').img_name.count().sort_values(ascending=False).idxmin()
print(min_agegroup)

13-15


In [120]:
# Get racial distribution of number of images in minority age group
minority_agedf = ylfw_witha[ylfw_witha.children_agegroup == min_agegroup]
minority_agedf.groupby('ethnicity').img_name.count().sort_values(ascending=False)

ethnicity
African      164
Indian       158
Caucasian    117
Asian        112
Name: img_name, dtype: int64

In [121]:
minority_agedf

,img_name,Age,ethnicity,id_class,children_agegroup,im_id,face_id
22,African_634_22,15,African,634,13-15,African_634,NaN
60,Caucasian_346_12,14,Caucasian,346,13-15,Caucasian_346,NaN
62,African_699_1,13,African,699,13-15,African_699,NaN
164,African_412_47,13,African,412,13-15,African_412,NaN
168,Asian_489_46,13,Asian,489,13-15,Asian_489,NaN
...,...,...,...,...,...,...,...
9897,m.05lq0j_0003,14,Asian,05lq0j,13-15,m.05lq0j,3.0
9906,m.05fwlf_0002,13,Asian,05fwlf,13-15,m.05fwlf,2.0
9913,m.0bmd0_6_0002,15,Asian,0bmd0_6,13-15,m.0bmd0_6,2.0
9915,m.0bmd0_6_0001,13,Asian,0bmd0_6,13-15,m.0bmd0_6,1.0


I.e. Remove samples in the other age groups such that they have approximately the same amount of images and same racial distribution. This is done by sampling the same amount of images within each ethnicity group. 

In [122]:
print("min age group: ", min_agegroup, "\nnumber of images: ", len(minority_agedf),
      "\n\nracial distribution:", minority_agedf.groupby('ethnicity').img_name.count().sort_values(ascending=False))

min age group:  13-15 
number of images:  551 

racial distribution: ethnicity
African      164
Indian       158
Caucasian    117
Asian        112
Name: img_name, dtype: int64


OBS inden dette: lav ethnicity dist for hver age group, for måske også og se hvilken der er mest balanced, og så balancér ift denne?? eller er det too much? 

In [123]:
minority_agedf

,img_name,Age,ethnicity,id_class,children_agegroup,im_id,face_id
22,African_634_22,15,African,634,13-15,African_634,NaN
60,Caucasian_346_12,14,Caucasian,346,13-15,Caucasian_346,NaN
62,African_699_1,13,African,699,13-15,African_699,NaN
164,African_412_47,13,African,412,13-15,African_412,NaN
168,Asian_489_46,13,Asian,489,13-15,Asian_489,NaN
...,...,...,...,...,...,...,...
9897,m.05lq0j_0003,14,Asian,05lq0j,13-15,m.05lq0j,3.0
9906,m.05fwlf_0002,13,Asian,05fwlf,13-15,m.05fwlf,2.0
9913,m.0bmd0_6_0002,15,Asian,0bmd0_6,13-15,m.0bmd0_6,2.0
9915,m.0bmd0_6_0001,13,Asian,0bmd0_6,13-15,m.0bmd0_6,1.0


In [124]:
# get dist of images per ID in min group
minority_agedf.groupby("im_id").agg({'img_name': ['count']}).describe()

,img_name
,count
count,428.000000
mean,1.287383
std,0.622851
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,4.000000


In [125]:
print("Average number of ims per id")
minority_agedf.groupby("im_id").agg({'img_name': ['count']}).mean()

Average number of ims per id


img_name  count    1.287383
dtype: float64

In [126]:
minority_agedf

,img_name,Age,ethnicity,id_class,children_agegroup,im_id,face_id
22,African_634_22,15,African,634,13-15,African_634,NaN
60,Caucasian_346_12,14,Caucasian,346,13-15,Caucasian_346,NaN
62,African_699_1,13,African,699,13-15,African_699,NaN
164,African_412_47,13,African,412,13-15,African_412,NaN
168,Asian_489_46,13,Asian,489,13-15,Asian_489,NaN
...,...,...,...,...,...,...,...
9897,m.05lq0j_0003,14,Asian,05lq0j,13-15,m.05lq0j,3.0
9906,m.05fwlf_0002,13,Asian,05fwlf,13-15,m.05fwlf,2.0
9913,m.0bmd0_6_0002,15,Asian,0bmd0_6,13-15,m.0bmd0_6,2.0
9915,m.0bmd0_6_0001,13,Asian,0bmd0_6,13-15,m.0bmd0_6,1.0


In [127]:
minority_agedf

,img_name,Age,ethnicity,id_class,children_agegroup,im_id,face_id
22,African_634_22,15,African,634,13-15,African_634,NaN
60,Caucasian_346_12,14,Caucasian,346,13-15,Caucasian_346,NaN
62,African_699_1,13,African,699,13-15,African_699,NaN
164,African_412_47,13,African,412,13-15,African_412,NaN
168,Asian_489_46,13,Asian,489,13-15,Asian_489,NaN
...,...,...,...,...,...,...,...
9897,m.05lq0j_0003,14,Asian,05lq0j,13-15,m.05lq0j,3.0
9906,m.05fwlf_0002,13,Asian,05fwlf,13-15,m.05fwlf,2.0
9913,m.0bmd0_6_0002,15,Asian,0bmd0_6,13-15,m.0bmd0_6,2.0
9915,m.0bmd0_6_0001,13,Asian,0bmd0_6,13-15,m.0bmd0_6,1.0


In [128]:
etnicities = list(ylfw_witha.ethnicity.unique())

# Save variables of frequency within each ethnicity
minority_etnicity_dist = minority_agedf.groupby('ethnicity').img_name.count().sort_values(ascending=False)
minority_n_ids = len(minority_agedf) # kommer automatisk at nå ca samme fordeling


# Get this distribution of frequency in ethnicities from each age group
ylfw_witha_balanced = minority_agedf.copy()
for agegroup in age_labels:
    if agegroup != min_agegroup:
        agegroup_df = ylfw_witha[ylfw_witha.children_agegroup == agegroup]
        nunique_ids = agegroup_df.im_id.nunique()
        #get number of ids based on percentage dist of the minority age group
        for e in etnicities:
            freq_e = minority_etnicity_dist[e]

            # randomly sample the images from the current age group from this ethnicity
            age_ethn_df = agegroup_df[agegroup_df.ethnicity == e]
            sample_eids = age_ethn_df.sample(n=freq_e, random_state=42)

            # add theese to the minority dataset
            ylfw_witha_balanced = pd.concat([ylfw_witha_balanced, sample_eids], ignore_index=True)


Check if balanced

In [129]:
# Age groups
ylfw_witha_balanced.children_agegroup.value_counts()

children_agegroup
13-15    551
0-3      551
4-6      551
7-9      551
10-12    551
16-18    551
Name: count, dtype: int64

In [130]:
# Race distribution
print("Min age group distribution", minority_agedf.groupby('ethnicity').img_name.count().sort_values(ascending=False))
print("\n0-3\n",ylfw_witha_balanced[ylfw_witha_balanced.children_agegroup == "0-3"].groupby('ethnicity').img_name.count().sort_values(ascending=False))
print("\n16-18",ylfw_witha_balanced[ylfw_witha_balanced.children_agegroup == "16-18"].groupby('ethnicity').img_name.count().sort_values(ascending=False))

Min age group distribution ethnicity
African      164
Indian       158
Caucasian    117
Asian        112
Name: img_name, dtype: int64

0-3
 ethnicity
African      164
Indian       158
Caucasian    117
Asian        112
Name: img_name, dtype: int64

16-18 ethnicity
African      164
Indian       158
Caucasian    117
Asian        112
Name: img_name, dtype: int64


In [131]:
ylfw_witha_balanced.children_agegroup.value_counts()

children_agegroup
13-15    551
0-3      551
4-6      551
7-9      551
10-12    551
16-18    551
Name: count, dtype: int64

In [132]:
ylfw_witha_balanced.groupby("im_id").count().sort_values(by="img_name", ascending=False)

,img_name,Age,ethnicity,id_class,children_agegroup,face_id
im_id,,,,,,
Indian_484,8,8,8,8,8,0
Indian_577,7,7,7,7,7,0
Indian_602,7,7,7,7,7,0
Indian_531,7,7,7,7,7,0
Indian_667,7,7,7,7,7,0
...,...,...,...,...,...,...
Caucasian_1115,1,1,1,1,1,0
Caucasian_1112,1,1,1,1,1,0
Caucasian_1108,1,1,1,1,1,0


In [133]:
ylfw_witha_balanced.to_csv('../data/child_balanced.csv', index=False)

## 2. Balance Data in adult group and children's group


Mht race?

In [134]:
# Racial distribution
ylfw_witha_balanced.ethnicity.value_counts(normalize=True)

ethnicity
African      0.297641
Indian       0.286751
Caucasian    0.212341
Asian        0.203267
Name: proportion, dtype: float64

In [135]:
# Remove childrens age group from adult data
a_df = pd.read_csv('../data/raw_full/raw_rfw_df.csv')
a_df = a_df[a_df.Age > 18]

# Check racial distribution
a_df.ethnicity.value_counts(normalize=True)

ethnicity
Indian       0.256337
African      0.255932
Caucasian    0.250645
Asian        0.237086
Name: proportion, dtype: float64

In [136]:
etnicities = list(ylfw_witha_balanced.ethnicity.unique())

# Save variables of frequency within each ethnicity
etnicity_dist = ylfw_witha_balanced.groupby('ethnicity').img_name.count().sort_values(ascending=False)
a_balanced = pd.DataFrame()
for e in etnicities:
    freq_e = etnicity_dist[e]

    # randomly sample the images from the current age group from this ethnicity
    age_ethn_df = a_df[a_df.ethnicity == e]
    sample_eids = age_ethn_df.sample(n=freq_e, random_state=42)

    # add theese to the minority dataset
    a_balanced = pd.concat([a_balanced, sample_eids], ignore_index=True)


In [137]:
a_balanced.groupby('ethnicity').img_name.count().sort_values(ascending=False)

ethnicity
African      984
Indian       948
Caucasian    702
Asian        672
Name: img_name, dtype: int64

In [138]:
ylfw_witha_balanced.groupby('ethnicity').img_name.count().sort_values(ascending=False)


ethnicity
African      984
Indian       948
Caucasian    702
Asian        672
Name: img_name, dtype: int64

In [139]:
a_balanced

,img_name,im_id,ethnicity,id_class,face_id,Age,children_agegroup
0,m.0fw3k1_0002,m.0fw3k1,African,0fw3k1,2,20,NaN
1,m.04zvg12_0002,m.04zvg12,African,04zvg12,2,24,NaN
2,m.05mrf5k_0002,m.05mrf5k,African,05mrf5k,2,28,NaN
3,m.07b34k_0003,m.07b34k,African,07b34k,3,44,NaN
4,m.02xgm5_0002,m.02xgm5,African,02xgm5,2,34,NaN
...,...,...,...,...,...,...,...
3301,m.0h949qw_0001,m.0h949qw,Indian,0h949qw,1,22,NaN
3302,m.0bhcm0z_0003,m.0bhcm0z,Indian,0bhcm0z,3,32,NaN
3303,m.0bgrs7_0002,m.0bgrs7,Indian,0bgrs7,2,20,NaN
3304,m.0b4y2t_0003,m.0b4y2t,Indian,0b4y2t,3,38,NaN


In [140]:
a_balanced.to_csv('../data/adults_balanced.csv', index=False)